In [87]:
import base64
import requests
import dotenv
import os
import pandas as pd

## OCR with openai

In [50]:
dotenv_path = os.path.join(os.getcwd(), ".env")
dotenv.load_dotenv(dotenv_path)
api_key = os.getenv("OPENAI_API_KEY")


# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


# Path to your file
image_path = "data\example_auszug.png"

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {"Content-Type": "application/json", "Authorization": f"Bearer {api_key}"}

payload = {
    "model": "gpt-4-vision-preview",
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": """
Transcribe this file with correct formatting of tables. """,
                },
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}},
            ],
        }
    ],
    "max_tokens": 2000,
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)


<Response [200]>


In [93]:
print(response.json()["choices"][0]["message"]["content"])


Certainly! Below is the transcription of the table content from the given file, with the formatting replicated as closely as possible to plain text:

```
| Datum      | Wert    | Erläuterung                                  | Betrag EUR |
|------------|---------|----------------------------------------------|------------|
| 04.05.2016 | 04.05.2016 | Kontostand am 29.04.2016, Auszug Nr. 2    |            |
|            |         | SOLLZINSEN                   | 720,00-       |
|            |         | GUTSCHRIFT SB-TAGESGELD      | 720,00        |
|------------|---------|----------------------------------------------|------------|
| 04.05.2016 | 04.05.2016 | SB-TAGESGELD                  | 1.440,00-   | 
|            |         | ZINSEN TZR X.XXX,00           |              |
|            |         | ZINSABSCHLUSS                 |              |
|            |         | GUTSCHRIFT TZR                |              |
|            |         | SOLLZINSEN                    |              |

In [ ]:
# example raw table string
table_str = """
| Datum    | Wert  | Erläuterung                                                    | Betrag EUR |
|----------|-------|----------------------------------------------------------------|------------|
| 04/05/16 |       | STORNO RSV KONTOABSCHLUSS SIEHE TEXTERGAENZUNG                 |     720,00- |
| 04/05/16 |       | STORNO TG. ABSCHLUSS SIEHE TEXTERGAENZUNG                      |     720,00- |
| 04/05/16 |       | JAHRESBEITRAG                                                  |   1.440,00- |
"""

lines = [line for line in table_str.split('\n') if line.strip()]

# The first line is the header
headers = [header.strip() for header in lines[0].split('|') if header.strip()]

# The rest are data lines
data_lines = lines[2:]  

data = {header: [] for header in headers}

# Populate the dictionary with data
for line in data_lines:
    values = [value.strip() for value in line.split('|')[1:-1]]
    for i, header in enumerate(headers):
        data[header].append(values[i])


df = pd.DataFrame.from_dict(data)
df